<a href="https://colab.research.google.com/github/dborgesm/Banking-Regulations/blob/master/Banking_Regulations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Banking Regulations

Here is the analysis and assessment of the capital requirement of a credit portfolio, all the definitions and calculations are based on Basel Committee on Banking Supervision and Office of The Superintendent of Financial Institutions documentations. The capital requirement calculated was for a bank which portfolio is composed by retail, mortgages and corporate exposures, the foundational and standard approach were used and the Basel III PDs and LGDs were applied,

In [0]:
#Installing packages
import pandas as pd
import numpy as np

In [0]:
!gdown https://drive.google.com/uc?id=1_HeP1GynSCws3oQEQ9xZTjCAy6B2tHzQ

Downloading...
From: https://drive.google.com/uc?id=1_HeP1GynSCws3oQEQ9xZTjCAy6B2tHzQ
To: /content/Dataset.xlsx
17.0MB [00:00, 46.6MB/s]


In [0]:
#Loading the data
bond_data = pd.read_excel('/content/Dataset.xlsx', sheet_name = 'Bond')
mortgage_data = pd.read_excel('/content/Dataset.xlsx', sheet_name = 'Mortgage')
rloan_data = pd.read_excel('/content/Dataset.xlsx', sheet_name = 'RetailLoan')
bond_yield = pd.read_excel('/content/Dataset.xlsx', sheet_name = 'Yields') #This excel sheet was added using the yields of the Bank of Canada from September 26th 2019

# Capital Adequacy under the Internal Ratings Based Approach (IRB)
## Calculating the Exposure at Default

## Bonds

For the corporate bonds portfolio, it was used the equivalent rate of a bond which pay yearly coupons, the exposure was calculated by bringing to present value the remaining coupons using the market rate yields ([Canada's bond rate](https://www.bankofcanada.ca/rates/interest-rates/canadian-bonds/)) multiplied by the total of bonds held and divided by 100 because the prices were in cents. Hence, the total EAD of our bond portfolio resulted in $52,599.23 million dollars.

In [0]:
# This function calculates the price of the yearly bond using the market yields
def bpv(pr, c, n, bh, r = bond_yield['Yield']):
    
    c = pr*(c/100)
    if round(n) < 1:
        
        tp = (pr / ((1+r[0])**n))
            
        return tp*bh/100
    else:
        f = n
        t = int(round(n))
        price = np.zeros(t+1)
        
        for i in range(t):
            if i + 1 <= 3:
                price[i] = np.pv(rate=r[0], nper=i+1, pmt=0, fv=-c)
                price[t] = (pr/((1+r[0])**f))
                
            elif (i+1>3) & (i+1<=5):
                price[i] = np.pv(rate=r[1], nper=i+1, pmt=0, fv=-c)
                price[t] = (pr/((1+r[1])**f))
                
            elif (i+1>5) & (i+1<=10):
                price[i] = np.pv(rate=r[2], nper=i+1, pmt=0, fv=-c)
                price[t] = (pr/((1+r[2])**f))
            else:
                price[i] = np.pv(rate=r[3], nper=i+1, pmt=0, fv=-c)
                price[t] = (pr/((1+r[3])**f))
        
        tp = np.sum(price)*bh/100       
        return tp

In [0]:
bond_data['EAD'] = bond_data.apply(lambda x : bpv(x['Principal'], x['YearlyRate'], x['Time2Maturity'],x['BondsHeld']),axis=1)

In [0]:
bond_data.describe()

,id,PurchasePrice,CurrentCoupon,CouponsRemaining,Time2Maturity,BondsHeld,YearlyRate,Principal,PD,EAD
count,178502.000000,178502.000000,178502.000000,178502.000000,178502.000000,1.785020e+05,178502.000000,178502.000000,178502.000000,1.785020e+05
mean,89251.500000,103.453157,5.644648,64.591786,6.586240,2.675009e+05,1.503375,105.383872,0.250443,2.946702e+05
std,51529.233213,9.851203,1.770356,21.482486,6.306710,7.727148e+05,0.865995,19.359049,0.283668,8.916276e+05
min,1.000000,1.062000,0.000000,0.000000,0.085313,7.000000e+00,0.010000,0.000000,0.000000,0.000000e+00
25%,44626.250000,99.445000,5.000000,50.000000,2.499073,1.000000e+04,0.750000,97.000000,0.015759,1.017936e+04
50%,89251.500000,102.327000,6.000000,62.000000,4.781210,2.500000e+04,1.500000,102.000000,0.125222,2.534558e+04
75%,133876.750000,107.176000,7.000000,76.000000,8.269435,1.000000e+05,2.250000,109.000000,0.423630,1.074680e+05
max,178502.000000,166.276000,14.000000,160.000000,84.159498,5.000001e+06,3.000000,1145.000000,0.999995,1.503478e+07


# Retail Loans

For the loans portfolio, considering monthly payments it was required to find the amount of payment by using the formula of an immediate annuity, the remaining payments were bring to present value converting the yearly rate to a monthly effective interest rate, hence the result the loan outstanding balance. By adding each one of these results the EAD of our loan portfolio resulted in $102.17 million dollars.

In [0]:
#Pricing the Retail Loans

def rloan_pricing(a, n, r, ci):
  # Transforming the annual rate
  i = ((r+1)**(1/12))-1
  # Finding the payment
  pmt = np.pmt(i, n, -a)
  
  # Calculating the price at time n-ci
  payment_series = np.repeat(pmt, n-ci+1)
  payment_series[0] -= pmt
  pvt = np.npv(i, payment_series)
  return pvt
    

In [0]:
rloan_data['EAD'] = rloan_data.apply(lambda x: rloan_pricing(x['OriginalAmount'], x['Term'], x['YearlyInterestRate'], x['CurrentInstallment']), axis=1)
rloan_data.describe()

,ID,OriginalAmount,Term,YearlyInterestRate,CurrentInstallment,PD,EAD
count,21537.00000,21537.000000,21537.000000,21537.000000,21537.000000,2.153700e+04,21537.000000
mean,10769.00000,8572.190231,27.656498,0.185789,13.354831,2.501155e-01,4743.769688
std,6217.34071,10780.933077,11.748903,0.080827,10.109312,2.832362e-01,6997.238285
min,1.00000,1137.000000,11.000000,0.050000,1.000000,1.176836e-13,72.267310
25%,5385.00000,2950.000000,22.000000,0.120000,6.000000,1.625598e-02,1293.889865
50%,10769.00000,5438.000000,23.000000,0.190000,11.000000,1.243691e-01,2641.813949
75%,16153.00000,9780.000000,35.000000,0.260000,18.000000,4.229269e-01,5469.588894
max,21537.00000,134750.000000,69.000000,0.320000,61.000000,9.998771e-01,126741.914134


## Mortgages

For the mortgages portfolio, considering monthly payments and converting the annual rate to a monthly effective interest rate, first the value of the payment was calculated using the formula of an immediate annuity, then the standing amount was calculated taking into consideration only the remaining installments minus the collateral, if the exposure was negative the minimum value considered was zero. Thus, the EAD of our mortgage portfolio resulted in $4,507.96 million dollars.

In [0]:
#Pricing the Mortgages

def mort_pricing(a, n, r, ci, co):
  
  # Transforming the annual rate
  i = ((r+1)**(1/12))-1
  # Finding the payment
  n = n*12
  pmt = np.pmt(i, n, -a)
  
  # Calculating the price at time n-ci
  payment_series = np.repeat(pmt, n+1-ci)
  payment_series[0] -= pmt
  pvt = max(np.npv(i, payment_series)-co, 0)

  return pvt
    

In [0]:
mortgage_data['EAD'] = mortgage_data.apply(lambda x: mort_pricing(x['OriginalAmount'], x['TermYears'], x['Rate'], x['CurrentInstallment'], x['Collateral_valuation']), axis=1)
mortgage_data.describe()

,ID,OriginalAmount,TermYears,Collateral_valuation,CurrentInstallment,Rate,PD,LGD,EAD
count,28463.000000,2.846300e+04,28463.000000,2.846300e+04,28463.000000,28463.000000,2.846300e+04,28463.000000,2.846300e+04
mean,14232.000000,1.079277e+06,25.096582,8.043405e+05,150.281594,0.074991,2.521387e-01,0.500315,1.425697e+05
std,8216.704692,1.197933e+06,11.204059,1.025011e+06,115.199717,0.031922,2.847395e-01,0.217920,3.904055e+05
min,1.000000,1.429020e+05,11.000000,1.257000e+03,1.000000,0.020000,7.142065e-13,0.005243,0.000000e+00
25%,7116.500000,3.813850e+05,20.000000,2.416890e+05,63.000000,0.048000,1.613579e-02,0.334014,0.000000e+00
50%,14232.000000,6.977920e+05,23.000000,4.735740e+05,124.000000,0.075000,1.264557e-01,0.499345,0.000000e+00
75%,21347.500000,1.257019e+06,34.000000,9.538715e+05,215.000000,0.103000,4.234764e-01,0.667212,1.197973e+05
max,28463.000000,1.673960e+07,68.000000,1.729625e+07,731.000000,0.130000,9.999378e-01,0.998240,1.132862e+07


## Expected Loss and Loss Given Default

The expected loss is the result of the EAD multiplied by the probability of default. 

#### Probability of Default
For the retail and corporate exposure, it was considered a minimum probability of default of 0.03%. 

#### Loss Given Default
For the mortgage portfolio the minimum LGD considered was 10% 

For the bond portfolio the minimum LGD considered was 45%

The PD and LGD floors can be found in the [Canadian OFSI Chapter 6](https://www.osfi-bsif.gc.ca/Eng/fi-if/rg-ro/gdn-ort/gl-ld/Pages/CAR19_chpt6.aspx)

In [0]:
# EL function and LGD function 
def floor_pd_lgd(x, minv=0.0003):
  if x < minv:
    return minv
  else:
    return x


In [0]:
# Adding a new column PD2, where if the min PD=0.03% otherwise it will be the one assigned
bond_data['PD2'] = bond_data.apply(lambda x: floor_pd_lgd(x['PD'], minv=0.0003), axis=1)
bond_data.describe()

,id,PurchasePrice,CurrentCoupon,CouponsRemaining,Time2Maturity,BondsHeld,YearlyRate,Principal,PD,EAD,PD2
count,178502.000000,178502.000000,178502.000000,178502.000000,178502.000000,1.785020e+05,178502.000000,178502.000000,178502.000000,1.785020e+05,178502.000000
mean,89251.500000,103.453157,5.644648,64.591786,6.586240,2.675009e+05,1.503375,105.383872,0.250443,2.946702e+05,0.250458
std,51529.233213,9.851203,1.770356,21.482486,6.306710,7.727148e+05,0.865995,19.359049,0.283668,8.916276e+05,0.283655
min,1.000000,1.062000,0.000000,0.000000,0.085313,7.000000e+00,0.010000,0.000000,0.000000,0.000000e+00,0.000300
25%,44626.250000,99.445000,5.000000,50.000000,2.499073,1.000000e+04,0.750000,97.000000,0.015759,1.017936e+04,0.015759
50%,89251.500000,102.327000,6.000000,62.000000,4.781210,2.500000e+04,1.500000,102.000000,0.125222,2.534558e+04,0.125222
75%,133876.750000,107.176000,7.000000,76.000000,8.269435,1.000000e+05,2.250000,109.000000,0.423630,1.074680e+05,0.423630
max,178502.000000,166.276000,14.000000,160.000000,84.159498,5.000001e+06,3.000000,1145.000000,0.999995,1.503478e+07,0.999995


In [0]:
# Adding a new column PD2, where if the min PD=0.03% otherwise it will be the one assigned
rloan_data['PD2'] = rloan_data.apply(lambda x: floor_pd_lgd(x['PD'], minv=0.0003), axis=1)

In [0]:
# Adding two new columns of PD and LGD to the mortgage data where min LGD=10% and min PD=0.03%
mortgage_data['PD2'] = mortgage_data.apply(lambda x: floor_pd_lgd(x['PD'], minv=0.0003), axis=1)
mortgage_data['LGD2'] = mortgage_data.apply(lambda x: floor_pd_lgd(x['LGD'], minv=0.1), axis=1)
mortgage_data.head()

,ID,OriginalAmount,TermYears,Collateral_valuation,CurrentInstallment,Rate,PD,LGD,EAD,PD2,LGD2
0,18101,2067863,35,678311,306,0.082,0.004728,0.497927,4.852612e+05,0.004728,0.497927
1,7835,309683,22,259913,166,0.117,0.176124,0.278921,0.000000e+00,0.176124,0.278921
2,26416,2998421,21,2218432,133,0.092,0.014447,0.570983,0.000000e+00,0.014447,0.570983
3,7304,483754,23,246967,152,0.032,0.168241,0.405557,1.378896e+04,0.168241,0.405557
4,14807,1808956,37,499590,93,0.114,0.006678,0.457563,1.264946e+06,0.006678,0.457563


### Calculating the Expected Loss

In [0]:
# Function to calculate EL 
def expected_loss(exp, pd, lgd):
  return exp*pd*lgd

In [0]:
# Calculating the EL of retail loans
rloan_data['EL'] = rloan_data.apply(lambda x: expected_loss(x['EAD'], x['PD2'],lgd=1), axis=1)
rloan_data['EL'].describe()

count    21537.000000
mean      1188.772282
std       2913.095540
min          0.027615
25%         35.795763
50%        278.113234
75%       1127.149569
max      76874.694465
Name: EL, dtype: float64

In [0]:
# Calculating the EL of mortgages
mortgage_data['EL'] = mortgage_data.apply(lambda x: expected_loss(x['EAD'], x['PD2'],x['LGD2']), axis=1)
mortgage_data['EL'].describe()


count    2.846300e+04
mean     1.775310e+04
std      7.905192e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      2.986642e+03
max      2.402081e+06
Name: EL, dtype: float64

In [0]:
# Calculating EL of corporate bonds
bond_data['EL'] = bond_data.apply(lambda x: expected_loss(x['EAD'], x['PD2'],lgd=0.45), axis=1)
bond_data['EL'].describe()

count    1.785020e+05
mean     3.313238e+04
std      1.552920e+05
min      0.000000e+00
25%      1.709387e+02
50%      1.418027e+03
75%      7.527994e+03
max      4.786673e+06
Name: EL, dtype: float64

## Basel III Capital Requirements

Recalling the last lecture, the equation for the capital requirement of any operation is:

$$
K = LGD \cdot \left\{ N\left( \sqrt{\frac{1}{1-R}} \cdot N^{-1}(PD) + \sqrt{\frac{R}{1-R}} \cdot N^{-1}(0.999) \right) - PD \right\} \left( \frac{1 + (M + 2.5)b}{1 - 1.5b}\right)
$$

The values of $b$ and $M$ will be variable for bonds, but for retail and mortgages the maturity is fixed at 1, and the b term dissapears. The correlations are given by the regulation:

- Mortgages: $R = 0.15$
- Revolving: $R = 0.04$
- Other retail: $R = 0.03 \left( \frac{1 - e^{-35PD}}{1 - e^{-35}} \right) + 0.16 \left( 1 - \frac{1 - e^{-35PD}}{1 - e^{-35}} \right)$
- Corporate and sovereign exposures $ R = 0.12 \left( \frac{1 - e^{-50PD}}{1 - e^{-50}} \right) + 0.24 \left( 1 - \frac{1 - e^{-50PD}}{1 - e^{-50}} \right)$

With this we can calculate the capital requirements and the Risk Weighted Assets (RWA) for this portfolio. Let's start implementing the capital requirement function. Note that we require the cumulative normal distribution  and its inverse functions. For this we will use numpy's sister package [```scipy```](https://scipy.org/) which includes all (traditional) statistical models and quantities for classic stats (not analytics!) in its subpackage [```stats```](https://docs.scipy.org/doc/scipy/reference/tutorial/stats.html).

Within the package stats, we find the statistical distribution we need: [```norm```](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html#scipy.stats.norm), the standard normal. Within it, we can call the cumulative function (```norm.cdf```) and the inverse function, ```norm.ppf``` which stands for *[percent point function](https://stackoverflow.com/questions/20626994/how-to-calculate-the-inverse-of-the-normal-cumulative-distribution-function-in-p)*.

In [0]:
# Capital requirement for bonds
def capital_requirement_bond(PD, LGD, M):
  import numpy as np  
  from scipy.stats import norm
  R =  0.12 * ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) 
  R += 0.24 * (1 - ( (1 - np.exp(-50 * PD)) / (1 - np.exp(-50)) ) )
  b = (0.11852 - 0.05478*np.log(PD))**2
  bm = (1+(M-2.5)*b)/(1-(1.5*b))
  K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
               np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
  K *= LGD * bm 
    
  return(K)


In [0]:
# Capital requirement for mortgages and retail loans 
def capital_requirement_retail(PD, LGD, t = 'retail'):
  import numpy as np
  from scipy.stats import norm
  if t == "retail":
    R =  0.03 * ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) 
    R += 0.16 * (1 - ( (1 - np.exp(-35 * PD)) / (1 - np.exp(-35)) ) )
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
                 np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD
    
  else:
    R =  0.15 
    K = norm.cdf(np.sqrt( (1 - R) ** (-1) ) * norm.ppf(PD) + 
                 np.sqrt( R / (1 - R) ) * norm.ppf(0.999) ) - PD
    K *= LGD    
  return(K) 

In [0]:
# Calculating the K for every bond
bond_data['CapitalReq'] = bond_data.apply(lambda x : capital_requirement_bond(x['PD2'],0.45, x['Time2Maturity']), axis = 1)
bond_data['CapitalReq'].describe()


count    178502.000000
mean          0.144868
std           0.081199
min           0.000002
25%           0.083299
50%           0.150237
75%           0.197137
max           0.922894
Name: CapitalReq, dtype: float64

In [0]:
# Calculating the K for every retail loan
rloan_data['CapitalReq'] = rloan_data.apply(lambda x : capital_requirement_retail(x['PD2'],1,'retail'), axis = 1)
rloan_data['CapitalReq'].describe()


In [0]:
# Calculating the K for every mortgage
mortgage_data['CapitalReq'] = mortgage_data.apply(lambda x : capital_requirement_retail(x['PD2'],x['LGD2'],'mortgage'), axis = 1)
mortgage_data['CapitalReq'].describe()

Now that the Capital Requirement was calculated, the Risk Weighted Assets can be caculated. 

$RWA = 12.5*K*EAD$

For this exercise the capital requirement that the bank must satisfied according to OSFI is 11.5% of its risk weighted assets resulting in a total of $12,820.23 million CAD. 

In [0]:
# Calculating the RWA

rwa_rloan = 12.5*np.dot(rloan_data['CapitalReq'], rloan_data['EAD'])
rwa_mortgage = 12.5*np.dot(mortgage_data['CapitalReq'], mortgage_data['EAD'])
rwa_bond = 12.5*np.dot(bond_data['CapitalReq'], bond_data['EAD'])

#.115*(rwa_rloan+rwa_mortgage+rwa_bond)
rwa = pd.DataFrame([rwa_rloan, rwa_mortgage, rwa_bond], columns=['RWA'], index=['Loan', 'Mortgage','Bond'])
rwa['CapitalAdequacy'] = [rwa_rloan*.115,rwa_mortgage*.115,rwa_bond*.115]


In [0]:
y = pd.DataFrame([np.sum(rwa['RWA']), np.sum(rwa['CapitalAdequacy'])], columns=['SumTotal'],index=['RWA','CapitalAdequacy'])
y = y.transpose()
rwa.append(y)


,RWA,CapitalAdequacy
Loan,1.600855e+08,1.840983e+07
Mortgage,6.484765e+09,7.457480e+08
Bond,1.048354e+11,1.205607e+10
SumTotal,1.114803e+11,1.282023e+10


In [0]:
rwa['EL'] = [np.sum(rloan_data['EL']), np.sum(mortgage_data['EL']), np.sum(bond_data['EL'])]
rwa['EAD'] = [np.sum(rloan_data['EAD']), np.sum(mortgage_data['EAD']), np.sum(bond_data['EAD'])]
rwa

,RWA,CapitalAdequacy,EL,EAD
Loan,1.600855e+08,1.840983e+07,2.560259e+07,1.021666e+08
Mortgage,6.484765e+09,7.457480e+08,5.053065e+08,4.057962e+09
Bond,1.048354e+11,1.205607e+10,5.914197e+09,5.259923e+10


### Unexpected Loss

The losses above the expected levels are known as unexpected loss. The unexpected loss is the result of the EAD multiplied by the capital needed.

The total unexpected loss of the portfolio is $8,918.42 million dollars.

In [0]:
ul_rloan = np.dot(rloan_data['CapitalReq'], rloan_data['EAD'])
ul_mortgage = np.dot(mortgage_data['CapitalReq'], mortgage_data['EAD'])
ul_bond = np.dot(bond_data['CapitalReq'], bond_data['EAD'])
rwa['UL'] = [ul_rloan, ul_mortgage, ul_bond]
rwa

,RWA,CapitalAdequacy,EL,EAD,UL
Loan,1.600855e+08,1.840983e+07,2.560259e+07,1.021666e+08,1.280684e+07
Mortgage,6.484765e+09,7.457480e+08,5.053065e+08,4.057962e+09,5.187812e+08
Bond,1.048354e+11,1.205607e+10,5.914197e+09,5.259923e+10,8.386834e+09


In [0]:
y = pd.DataFrame([np.sum(rwa['RWA']), np.sum(rwa['CapitalAdequacy']), np.sum(rwa['EL']), np.sum(rwa['EAD']), np.sum(rwa['UL'])],
                 columns=['SumTotal'],index=['RWA','CapitalAdequacy','EL','EAD','UL'])
y = y.transpose()
rwa.append(y)

,RWA,CapitalAdequacy,EL,EAD,UL
Loan,1.600855e+08,1.840983e+07,2.560259e+07,1.021666e+08,1.280684e+07
Mortgage,6.484765e+09,7.457480e+08,5.053065e+08,4.057962e+09,5.187812e+08
Bond,1.048354e+11,1.205607e+10,5.914197e+09,5.259923e+10,8.386834e+09
SumTotal,1.114803e+11,1.282023e+10,6.445106e+09,5.675936e+10,8.918422e+09


# Capital Requirement using the Standard Approach

The method uses the multiplication of the EAD by the weights as specified in OSFI. For the loans the risk weight used was 75%, for the mortgages the risk weight was 35% and for the bonds the risk weight depended on their PD and rating giving in the coursework. Because the bank must satisfy 11.5% of its RWAs the capital requirement is $4,918.81 million CAD.

In [0]:
# Calculating the Capital Requirement using the Standard metod
# For the retail exposure a RWA of 75%

stdCA_rl = np.sum(rloan_data['EAD']*.75)
stdCA_rl


76624925.82612172

In [0]:
# For the mortgages exposure a RWA of 35%
stdCA_m = np.sum(mortgage_data['EAD']*.35)
stdCA_m

1420286758.8332229

In [0]:
# For the bonds exposure a RWA depends on the Raiting
# function that assigns the risk weight
def risk_weight(pd):
  
  if pd <= .01:
    return 0.2
  elif (pd>.01) & (pd<=0.12):
    return 0.5
  elif (pd > .12) & (pd <=0.52):
    return 1
  elif (pd >.52):
    return 1.5

In [0]:
bond_data['SRW'] = bond_data.apply(lambda x: risk_weight(x['PD']), axis=1)
stdCA_b = np.dot(bond_data['SRW'], bond_data['EAD'])

In [0]:
# Capital Adequacy
(stdCA_b + stdCA_rl + stdCA_m)*.115

In [0]:
# Capital Adequacy
((stdCA_b + stdCA_rl + stdCA_m)*.115)/1000000

4918.812462978633